[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JiayueMa/CUS754_SPRING2024_JiayueMa/blob/Final_Project/codes.ipynb)

Connect with the google drive

In [3]:
#You can find the data set at https://drive.google.com/drive/folders/18ir9EY9nvsiTh3u8rKDCx7EYodoxBgLF?usp=sharing 
# or in the github folder


from google.colab import drive
drive.mount('/content/drive')

import os


photo_dir = '/content/drive/MyDrive/Colab/project/target'

photos = [os.path.join(photo_dir, f) for f in os.listdir(photo_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
print(f"Total photos: {len(photos)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total photos: 1943


Training the model

In [4]:
import torch
import torch.nn as nn
import cv2
from PIL import Image
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
from google.colab.patches import cv2_imshow
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

# init
model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT).eval()
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# get the features of the people
def extract_target_features(model, loader):
    model.eval()
    with torch.no_grad():
        features_list = []
        for inputs, _ in loader:
            inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
            outputs = model(inputs)
            features_list.append(outputs)
        target_features = torch.cat(features_list).mean(0).unsqueeze(0)
    return target_features

# load the pic and features
target_data_dir = '/content/drive/MyDrive/Colab/project'
target_dataset = datasets.ImageFolder(root=target_data_dir, transform=transform)
target_loader = DataLoader(target_dataset, batch_size=1, shuffle=False)
target_features = extract_target_features(model, target_loader)


Testing the model

In [ ]:

# operate the video
cap = cv2.VideoCapture('/content/drive/MyDrive/Colab/videos/trump_video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter('path_to_your_output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (frame_width, frame_height))

# load the machine of face detecting
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            face_img_pil = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))

            if face_img_pil.mode == 'P':
                face_img_pil = face_img_pil.convert('RGBA')

            face_tensor = transform(face_img_pil).unsqueeze(0).to('cuda' if torch.cuda.is_available() else 'cpu')

            with torch.no_grad():
                face_features = model(face_tensor)
                face_features = face_features.squeeze()

            target_features = target_features.squeeze()
            similarity = F.cosine_similarity(face_features.unsqueeze(0), target_features.unsqueeze(0))
            detected = similarity.item() > 0.75

            if detected:
                cv2.circle(frame, (x + w//2, y + h//2), max(w, h)//2, (0, 255, 0), 4)

        out.write(frame)
        #cv2_imshow(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()
